## 딥러닝 활용(3)
### : 시퀀스 배열로 다루는 순환 신경망 ( RNN : Recurrent Neural Network )
* 여러 개의 데이터가 순서대로 입력됐을 때 앞서 입력받은 데이터를 잠시 기억해 놓는 방법
* 그 중에서도 LSTM(Long Short Term Memory)를 가장 널리 사용함

### >> LSTM을 이용한 로이터 뉴스 카테고리 분류하기
-> 긴 텍스트를 읽고 이 데이터가 어떤 의미를 가졌는지 카테고리로 분류하기

In [2]:
#데이터 준비하기
from keras.datasets import reuters

(X_train, Y_train),(X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

2113536/2110848 [==============================] - 3s 1us/step


In [4]:
#데이터 확인
import numpy as np
category = np.max(Y_train)+1
print(category,"카테고리")
print(len(X_train),'학습용 뉴스 기사')
print(len(X_test),'테스트용 뉴스 기사')
print(X_train[0])

46 카테고리
8982 학습용 뉴스 기사
2246 테스트용 뉴스 기사
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [6]:
#각 기사의 단어 수 맞추기 
from keras.preprocessing import sequence
from keras.utils import np_utils

x_train = sequence.pad_sequences(X_train, maxlen=100)
x_test = sequence.pad_sequences(X_test, maxlen=100)

y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

In [10]:
#모델의 설정
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

model = Sequential()
model.add(Embedding(1000,100))
model.add(LSTM(100,activation='tanh'))
model.add(Dense(46, activation = 'softmax'))

#모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#모델 실행
history = model.fit(x_train, y_train, batch_size = 100, epochs=20, validation_data = (x_test, y_test))

print("\nTest Accuracy: %.4f"%(model.evaluate(x_test,y_test)[1]))

Epoch 1/20
90/90 [==============================] - 20s 224ms/step - loss: 2.5946 - accuracy: 0.3480 - val_loss: 2.4158 - val_accuracy: 0.3620
Epoch 2/20
90/90 [==============================] - 19s 207ms/step - loss: 2.1542 - accuracy: 0.4447 - val_loss: 1.9335 - val_accuracy: 0.5031
Epoch 3/20
90/90 [==============================] - 19s 207ms/step - loss: 1.8775 - accuracy: 0.5157 - val_loss: 1.8553 - val_accuracy: 0.5467
Epoch 4/20
90/90 [==============================] - 19s 208ms/step - loss: 1.8656 - accuracy: 0.5342 - val_loss: 1.8045 - val_accuracy: 0.5552
Epoch 5/20
90/90 [==============================] - 19s 210ms/step - loss: 1.7142 - accuracy: 0.5609 - val_loss: 1.7286 - val_accuracy: 0.5744
Epoch 6/20
90/90 [==============================] - 19s 206ms/step - loss: 1.6477 - accuracy: 0.5796 - val_loss: 1.6778 - val_accuracy: 0.5815
Epoch 7/20
90/90 [==============================] - 18s 203ms/step - loss: 1.6015 - accuracy: 0.5897 - val_loss: 1.6410 - val_accuracy: 0.5824

### LSTM과 CNN의 조합으로 영화리뷰 분류하기

In [15]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#seed값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

#학습, 테스트 데이터 준비 및 전처리
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=5000)

x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

#모델 설정 밑 컴파일
model = Sequential()
model.add(Embedding(5000,100))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

#모델 실행
history=model.fit(x_train, y_train, batch_size=100, epochs=5, validation_data=(x_test, y_test))

print('\n Test Accuracy: %.4f'%(model.evaluate(x_test, y_test)[1]))

Epoch 1/5
250/250 [==============================] - 27s 109ms/step - loss: 0.4485 - accuracy: 0.7742 - val_loss: 0.3673 - val_accuracy: 0.8382
Epoch 2/5
250/250 [==============================] - 26s 105ms/step - loss: 0.2961 - accuracy: 0.8751 - val_loss: 0.3338 - val_accuracy: 0.8562
Epoch 3/5
250/250 [==============================] - 26s 104ms/step - loss: 0.2540 - accuracy: 0.8944 - val_loss: 0.3249 - val_accuracy: 0.8597
Epoch 4/5
250/250 [==============================] - 28s 110ms/step - loss: 0.2164 - accuracy: 0.9157 - val_loss: 0.3286 - val_accuracy: 0.8556
Epoch 5/5
782/782 [==============================] - 11s 14ms/step - loss: 0.3483 - accuracy: 0.8555

 Test Accuracy: 0.8555
